In [1]:

import numpy as np

In [3]:

y = np.array([[0.1, 0.2, 0.1, 0.4, 0.6], # p(blank | x)
              [0.7, 0.3, 0.1, 0.3, 0.2], # p(l1 | x)
              [0.1, 0.1, 0.4, 0.2, 0.1], # p(l2 | x)
              [0.1, 0.4, 0.4, 0.1, 0.1], # p(l3 | x)
              ]) 
#              t=1  t=2  t=3  t=4  t=5

labels = [1, 0, 2, 1] 

def forward(y, l, blank_idx=0, normalize=True):
    #alpha_t(s) is alpha[s, t]
    V, T = y.shape
    lenl_prime = 2*len(l) + 1
    # insert every other label with a blank
    l_prime = [blank_idx] + [l[i//2] if i % 2 == 1 else blank_idx for i in range(1, lenl_prime)] #TODO: vectorize
    alpha = np.zeros((lenl_prime, T))
    # Initialization
    alpha[0, 0] = y[blank_idx, 0] # blank at t=0
    alpha[1, 0] = y[l[0], 0] # first phoneme at t=0
    
    # Recursion
    for t in range(1, T):
        for s in range(lenl_prime):
            
            if s < lenl_prime - 2*(T-t) - 1:
                continue

            alpha_bar = alpha[s, t-1] + alpha[s-1, t-1]

            if l_prime[s] == blank_idx or l_prime[s-2] == l_prime[s]:
                alpha[s, t] = alpha_bar * y[l_prime[s], t]
            else:
                alpha[s, t] = (alpha_bar + alpha[s-2, t-1]) * y[l_prime[s], t]

            if normalize:
                C = np.sum(alpha[:, t])
                alpha[s, t] = alpha[s, t] / C # alpha_hat_t(s)
    
    return alpha

def backward(y, l, blank_idx=0, normalize=True):
    V, T = y.shape
    lenl_prime = 2*len(l) + 1
    # insert every other label with a blank
    l_prime = [blank_idx] + [l[i//2] if i % 2 == 1 else blank_idx for i in range(1, lenl_prime)] #TODO: vectorize
    beta = np.zeros((lenl_prime, T))

    # Initialization
    beta[lenl_prime-1, T-1] = y[blank_idx, T-1] # blank at t=T
    beta[lenl_prime-2, T-1] = y[l[-1], T-1] # last phoneme at t=T

    # Recursion
    for t in range(T-2, -1, -1):
        for s in range(lenl_prime):

            if s > 2*t:
                continue

            beta_bar = beta[s, t+1] + beta[s+1, t+1]

            if l_prime[s] == blank_idx or l_prime[s+2] == l_prime[s]:
                beta[s, t] = beta_bar * y[l_prime[s], t]
            else:
                beta[s, t] = (beta_bar + beta[s+2, t+1]) * y[l_prime[s], t]

            if normalize:
                D = np.sum(beta[:, t])
                beta[s, t] = beta[s, t] / (D + 1e-20)

    return beta




def compute_ctc_loss(y, l, t, blank_idx=0): # do we do this for all t's or what?
    # eq. 14
    alpha = forward(y, l, blank_idx)
    beta = backward(y, l, blank_idx)
    lenl_prime = 2*len(l) + 1
    l_prime = [blank_idx] + [l[i//2] if i % 2 == 1 else blank_idx for i in range(1, lenl_prime)] #TODO: vectorize
    probabilities = alpha * beta
    probabilities_t = probabilities[:, t]
    
    p_l_x = np.sum(probabilities_t / y[l_prime, t])

    return np.log(p_l_x)

print(compute_ctc_loss(y, labels, -1)) # this?


def compute_ctc_loss(y, l, blank_idx=0): # do we do this for all t's or what?
    # eq. 8
    alpha = forward(y, l, blank_idx)
    p_l_x = alpha[-1, -1] + alpha[-2, -1]
    return np.log(p_l_x)

print(compute_ctc_loss(y, labels)) # or this?

# Note: just doing the forward is enough to claculate the loss, but we need the backward for the gradient 
# (hence the same result)

-7.788351019734566
-7.788351019734566
